In [9]:
DATAPATH = "/mnt/cloud/lithuanian-asr/data"

In [1]:
!pip install pympi-ling

In [2]:
import pympi
from pathlib import Path
from pydub import AudioSegment

In [5]:
def get_eaf_data(filename):
    filepath = Path(filename)
    base = filepath.stem
    eaf = pympi.Elan.Eaf(filename)
    tiers = []

    if not 'speech' in eaf.tiers:
        print("Error reading file", filename)
        return []

    for tier in eaf.tiers['speech'][0].keys():
        current = {}
        id = f"{base}_{tier}"
        current["id"] = id
        data = eaf.tiers['speech'][0][tier]
        current["start"] = eaf.timeslots[data[0]]
        current["end"] = eaf.timeslots[data[1]]
        current["text"] = data[2]
        tiers.append(current)

    return tiers

In [3]:
def write_split_wavs(outdir, filename, data):
    outpath = Path(outdir)
    original = AudioSegment.from_wav(filename)
    for piece in data:
        outfile = outpath / f"{piece['id']}.wav"
        audio = original[piece["start"]:piece["end"]]
        audio.export(str(outfile), format="wav")


In [13]:
def append_to_tsv(tsv_file, data):
    with open(tsv_file, "a") as f:
        for item in data:
            f.write(f"{item['id']}\t{item['text']}\n")

In [6]:
SAMPLE = "/mnt/cloud/lithuanian-asr/data/R_RS_F3_AS113_01.eaf"
SAMPLE_WAV = SAMPLE.replace(".eaf", ".wav")
data = get_eaf_data(SAMPLE)
write_split_wavs("/tmp/foo", SAMPLE_WAV, data)

In [14]:
for eaf_file in Path(DATAPATH).glob("*.eaf"):
    data = get_eaf_data(eaf_file)
    wav_file = str(eaf_file).replace(".eaf", ".wav")
    write_split_wavs("/mnt/cloud/liepa-split", wav_file, data)
    append_to_tsv("/mnt/cloud/liepa-split/text.tsv", data)

KeyError: 'speech'